

Concentration



\begin{align}
	\Xi^{(3)}(x,y,s) =&
	\sum_{ (u_1,u_2) \in U_2}
	\lambda_0 \cdot ( x_{u_1} x_{u_2} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2} y_{t(u_1 u_2)} ) \nonumber\\&
	+  
	\lambda_1 \cdot
	\bigg[
	\left \lfloor J \right \rfloor^2 + \sum_{(u_{1}u_{2}u_{3}u_{4}) \in (U_2) ^ 2}y_{t(u_{1}u_{2})} y_{t(u_{3}u_{4})}+ \sum_{l_1 l_2 =0}^{N-1} 2^{l_1+l_2} s_{l_1} s_{l_2} \nonumber\\&
	+\sum_{(u_{1}u_{2}) \in U_2} (- 2  \left \lfloor J \right \rfloor) y_{t(u_{1}u_{2})}
	+ \sum_{(u_{1}u_{2},v) \in U_2 \times \mathbb{N}[0,N-1] } 2^{l+1} y_{t(u_{1}u_{2})} s_l \nonumber\\&
	+ \sum_{l=0}^{N-1}(-1)2^{l+1}  \left \lfloor J \right \rfloor s_l 
	\bigg]
\end{align}

$x \in \mathbb{B}^{nm}$,
$y \in \mathbb{B}^{n^2 m}$,
$s \in \mathbb{B}^{N}$

In [1]:
import numpy as np
import math
import gurobipy as gpy
from gurobipy import GRB

n=4; m=2
alpha = 0.95
J = n*n*(alpha + (1-alpha)/m)
Jfloor = math.floor(J)
dim_y = n*n*m
dim_s = math.floor(1+math.log2(Jfloor))
offset_y = n*m
offset_s = n*m + dim_y

##############  x   #        y      #      s
##############  nm  #       n^2m    #      N
# n=5; m=3; ##  15  #        75     #      4
dim_qubo =      n*m  +    dim_y      +    dim_s

Q = np.zeros([dim_qubo,dim_qubo])
c = 0

lambda0 = 10 #amplification of Rosenberg terms
lambda1 = 10 #amplification of linearized terms
lambda2 = 10 #amplification of BSM

print("number of counterparts: ",n)
print("number of classes: ",m)
print("alpha constant", alpha)
print("J constant", J)
print("dimension of the problem: ", dim_qubo)
print("lambda coefficients:")
print(lambda0, lambda1, lambda2, sep=', ')

number of counterparts:  4
number of classes:  2
alpha constant 0.95
J constant 15.6
dimension of the problem:  44
lambda coefficients:
10, 10, 10


\begin{align}
	U_2 = \{(u_1,u_2)\in \mathbb{N}[0,n^2m-1]^2|&u_r = (i_r-1)m+j-1,\ \text{for every}\ r\in\{1,2\},\
	\nonumber\\&
	(i_1,i_2)\in \mathbb{N}[1,n]^2 , \  j \in \mathbb{N}[1,m] \} .
\end{align}

In [2]:
u2 = []
for j in range (m):
	for i1 in range(n):
		for i2 in range(n):
			u2_1=( (i1+1) -1 )*m + (j+1) -1
			u2_2=( (i2+1) -1 )*m + (j+1) -1
			u2.append([u2_1,u2_2])

\begin{align}
	\sum_{ (u_1,u_2) \in U_2}
	\lambda_0 \cdot ( x_{u_1} x_{u_2} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2} y_{t(u_1 u_2)} ) \nonumber\\&
\end{align}

In [3]:
for u2_item in u2:
	if u2_item[0]==u2_item[1]:
		Q[ u2_item[0],u2_item[1] ] += lambda0
	else:
		Q[ u2_item[0],u2_item[1] ] += lambda0*0.5
		Q[ u2_item[1],u2_item[0] ] += lambda0*0.5

for u2_item in u2:
	t = u2.index([u2_item[0],u2_item[1]])
	Q[ t + offset_y , t + offset_y ] += lambda0*3

for u2_item in u2:
	t = u2.index([u2_item[0],u2_item[1]])
	Q[ u2_item[0] , t + offset_y ] += lambda0*0.5*(-2)
	Q[ t + offset_y , u2_item[0] ] += lambda0*0.5*(-2)

for u2_item in u2:
	t = u2.index([u2_item[0],u2_item[1]])
	Q[ u2_item[1] , t + offset_y ] += lambda0*0.5*(-2)
	Q[ t + offset_y , u2_item[1] ] += lambda0*0.5*(-2)

\begin{align}
	\lambda_1 \left \lfloor J \right \rfloor^2 + \sum_{(u_{1}u_{2}u_{3}u_{4}) \in (U_2) ^ 2}\lambda_1 y_{t(u_{1}u_{2})} y_{t(u_{3}u_{4})}+ \sum_{l_1 l_2 =0}^{N-1} \lambda_1  2^{l_1+l_2} s_{l_1} s_{l_2}
\end{align}

In [4]:
c += Jfloor*Jfloor*lambda1
for u2_item1 in u2:
	for u2_item2 in u2:
		t12 = u2.index([u2_item1[0],u2_item1[1]])
		t34 = u2.index([u2_item2[0],u2_item2[1]])
		if t12 == t34:
			Q[ t12 + offset_y , t12 + offset_y ] += lambda1
		else:
			Q[ t12 + offset_y , t34 + offset_y ] += lambda1*0.5
			Q[ t34 + offset_y , t12 + offset_y ] += lambda1*0.5
for l_1 in range(dim_s):
	for l_2 in range(dim_s):
		if l_1 == l_2:
			Q[ l_1 + offset_s , l_2 + offset_s ] += lambda1*math.pow(2,l_1+l_2)
		else:
			Q[ l_1 + offset_s , l_2 + offset_s ] += lambda1*math.pow(2,l_1+l_2)*0.5
			Q[ l_2 + offset_s , l_1 + offset_s ] += lambda1*math.pow(2,l_1+l_2)*0.5

\begin{align}
	\sum_{(u_{1}u_{2}) \in U_2} (- 2 \lambda_1 \left \lfloor J \right \rfloor) y_{t(u_{1}u_{2})}
	+ \sum_{(u_{1}u_{2},v) \in U_2 \times \mathbb{N}[0,N-1] }\lambda_1 2^{l+1} y_{t(u_{1}u_{2})} s_l
	+ \sum_{l=0}^{N-1}(-1)2^{l+1} \lambda_1 \left \lfloor J \right \rfloor s_l
\end{align}

In [5]:
for u2_item in u2:
	t = u2.index([u2_item[0],u2_item[1]])
	Q[ t + offset_y , t + offset_y ] += lambda1*(-2)*Jfloor
for u2_item in u2:
	for l in range (dim_s):
		t = u2.index([u2_item[0],u2_item[1]])
		Q[ t + offset_y , l + offset_s ] += lambda1*pow(2,l+1)*0.5
		Q[ l + offset_s , t + offset_y ] += lambda1*pow(2,l+1)*0.5
for l in range(dim_s):
	Q[ l + offset_s , l + offset_s ] += (-1)*lambda1*math.pow(2,l+1)*Jfloor

Binary staircase matrix

In [6]:
QQ = np.zeros([n*m,n*m])
cc=0
# penalty: "one counterpart per class"
for ii in range(n):
    for jj in range(m):
        tt = ii*m+jj
        QQ[tt][tt] += -1
    for jj in range(m-1):
        for kk in range(jj+1,m):
            tt = ii*m+jj
            rr = ii*m+kk
            QQ[tt][rr] += 1
            QQ[rr][tt] += 1
    cc += 1
# penalty: "first counterpart in first class"
for jj in range(1, m):
    QQ[jj][jj] += 1
    QQ[0][jj] -= 0.5
    QQ[jj][0] -= 0.5

# penalty: "last counterpart in the last class"
for jj in range(m-1):
    tt = (n-1)*m+jj
    QQ[tt][tt] += 1
    QQ[(n*m)-1][tt] -= 0.5
    QQ[tt][(n*m)-1] -= 0.5

# penalty: "penalize not permitted submatrix"
# a submatrix is
# [[x1, x1], [x3, x4]]
for ii in range(n-1):
    for jj in range(m-1):
        x1 = ii*m+jj
        x2 = x1+1
        x3 = (ii+1)*m+jj
        x4 = x3+1
        # add linear terms
        QQ[x1][x1] += 1
        QQ[x4][x4] += 1
        # add quadratic terms
        QQ[x1][x2] += 0.5
        QQ[x2][x1] += 0.5
        QQ[x1][x3] -= 0.5
        QQ[x3][x1] -= 0.5
        QQ[x1][x4] -= 1
        QQ[x4][x1] -= 1
        QQ[x2][x3] += 0.5
        QQ[x3][x2] += 0.5
        QQ[x2][x4] -= 0.5
        QQ[x4][x2] -= 0.5
        QQ[x3][x4] += 0.5
        QQ[x4][x3] += 0.5

In [7]:
c=c+cc*lambda2
QQ = QQ*lambda2
QQ=np.pad(QQ, ((0, dim_qubo-n*m), (0, dim_qubo-n*m)), mode='constant', constant_values=0)
Q = QQ+Q

In [8]:
# print("number of counterparts: ",n)
# print("number of classes: ",m)
# print("alpha constant", alpha)
# print("J  constant", J)
# print("dimension of the problem: ", dim_qubo)
# sol_nm = np.array(solution[:n*m])
# reshape_sol = sol_nm.reshape(n,m)
# print("solution:")
# print(reshape_sol)
# print("lambda coefficients:")
# print(lambda0, lambda1, lambda2, sep=', ')


In [9]:
def gurobi_solver(m, n, matrix, c, g_n_sol, g_fidelity):
    size = matrix.shape[0]
    # model definition
    qubo_model = gpy.Model("QCS")
    qubo_vars = qubo_model.addVars(size, vtype=GRB.BINARY, name="x")

    # cost function definition
    qubo_expr = gpy.QuadExpr()
    row_idxs, col_idxs = np.nonzero(matrix)
    for ii, jj in zip(row_idxs, col_idxs):
        qubo_expr.add(matrix[ii, jj] * qubo_vars[ii] * qubo_vars[jj])
    qubo_expr.addConstant(c)

    # add const function to the model
    qubo_model.setObjective(qubo_expr, GRB.MINIMIZE)

    # Setting solver parameters
    qubo_model.setParam("OutputFlag", 1) # verbosity
    qubo_model.setParam("Seed", 0)  # fix seed
    # qubo_model.setParam("TimeLimit", timelimit)
    
    # Search more than 1 solution
    num_max_solutions = g_n_sol
    if num_max_solutions > 1:
        qubo_model.setParam("PoolSolutions", num_max_solutions)
        qubo_model.setParam("PoolSearchMode", 2)
        qubo_model.setParam("PoolGap", g_fidelity)

    # Run the Gurobi QUBO optimization
    qubo_model.optimize()

    # Print result
    if qubo_model.Status in {GRB.OPTIMAL, GRB.SUBOPTIMAL}:
        if num_max_solutions == 1:
            solution = [int(qubo_vars[i].X) for i in range(size)]
            if len(solution) > m*n:
                solution = solution[:m*n]
            print("\nBest solution:\n", np.array(solution).reshape(n, m))
            print("Cost of the function:", qubo_model.ObjVal)
        else:
            # select all the solutions or num_max_solutions solutions
            nfound = min(qubo_model.SolCount, num_max_solutions)

            for sol_idx in range(nfound):
                qubo_model.setParam(GRB.Param.SolutionNumber, sol_idx)
                qubo_bitstring = np.array(
                    [int(qubo_vars[jj].Xn) for jj in range(size)]
                )
                if qubo_bitstring.shape[0] > m*n:
                    qubo_bitstring = qubo_bitstring[:m*n]
                print(f"solution {sol_idx+1}:\n{qubo_bitstring.reshape(n,m)}")
                print("Cost of the function:", qubo_model.PoolObjVal)
    else:
        print("No solutions found")


In [10]:
#(m, n, matrix, c, g_n_sol, g_fidelity):
gurobi_solver(m,n, Q, c, 1, 0)

Set parameter Username
Set parameter LicenseID to value 2685653
Set parameter OutputFlag to value 1
Set parameter Seed to value 0
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.3 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 44 columns and 0 nonzeros
Model fingerprint: 0x23b1c0b0
Model has 752 quadratic objective terms
Variable types: 0 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2290.0000000
Found heuristic solution: objective 20.0000000
Presolve time: 0.01s
Presolved: 708 rows, 752 columns, 2124 nonzeros
Variable types: 0 continuous, 752 integer (752 binary)

Root relaxation: o